### Modelado

En este notebook utilizaremos los conjuntos que hemos inspeccionado y adecuado para realizar modelos que nos ayude a analizar los mensajes de texto y detectar si se trata de **spam** o menos.

El siguiente script está dividido en los siguientes bloques:

- **BLOQUE A**: carga de datos preprocesados.

- **BLOQUE B**: entrenamiento y inferencia con distintos modelos de ML.

- **BLOQUE C**: entrenamiento y inferencia con una red neuronal.

- **BLOQUE D**: transfer learning.

In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report


from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer

2024-11-26 10:25:59.418709: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 10:26:00.958127: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 10:26:00.965374: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 10:26:02.847907: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### BLOQUE A: Carga de datos
Antes de comenzar, cargaremos los datos que han sido adecuados en nuestra fase anterior de preprocesado

In [2]:
# Carga textos vectorizados
with open('../data/x_train_vec.pkl', 'rb') as f:
    X_train_vec = pickle.load(f)

with open('../data/x_test_vec.pkl', 'rb') as f:
    X_test_vec = pickle.load(f)

In [3]:
# Carga conjuntos de las etiquetas
with open('../data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('../data/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

### BLOQUE B: Entrenamiento de distinto modelos de ML

#### Regresión logistica

La regresión logística es un método de clasificación que modela la probabilidad de eventos binarios. Utilizando la función sigmoide, asigna valores entre 0 y 1, facilitando la predicción de categorías, como positivo o negativos, spam o no spam, en aplicaciones prácticas.

In [4]:
# Creamos el modelo
log_model = LogisticRegression()

In [5]:
# Entrenamiento o ajuste del modelo con los datos de entrenamiento
log_model.fit(X_train_vec, y_train)

LogisticRegression()

In [6]:
# Predecimos sobre los datos de entrenamiento
y_pred_train = log_model.predict(X_train_vec)

# Mostramos el "classification report"
print('Resultados conjunto de entrenamiento:\n')
print(classification_report(y_train, y_pred_train))

Resultados conjunto de entrenamiento:

              precision    recall  f1-score   support

         ham       0.96      0.99      0.98       589
        spam       0.99      0.96      0.97       590

    accuracy                           0.97      1179
   macro avg       0.98      0.97      0.97      1179
weighted avg       0.98      0.97      0.97      1179



In [7]:
# Predecimos sobre los datos de test
y_pred_test = log_model.predict(X_test_vec)

# Mostramos el "classification report" y "accuracy"
accuracy = accuracy_score(y_test, y_pred_test)
print('Resultados conjunto de test:\n')
print(f'Accuracy: {accuracy:.2f}\n')
print(classification_report(y_test, y_pred_test))

Resultados conjunto de test:

Accuracy: 0.94

              precision    recall  f1-score   support

         ham       0.93      0.96      0.94       148
        spam       0.96      0.93      0.94       147

    accuracy                           0.94       295
   macro avg       0.94      0.94      0.94       295
weighted avg       0.94      0.94      0.94       295



#### Gradient boosting

¿Qué es Boosting?

Boosting es un meta-algoritmo de aprendizaje automático que reduce el sesgo y la varianza en un contexto de aprendizaje supervisado. Consiste en combinar los resultados de varios clasificadores débiles para obtener un clasificador robusto. Cuando se añaden estos clasificadores débiles, se hace de modo que éstos tengan diferente peso en función de la exactitud de sus predicciones. Tras añadir un clasificador débil, los datos cambian su estructura de pesos: los casos mal clasificados ganan peso y los que son clasificados correctamente pierden peso.

**Gradient Boosting (GB)** o Potenciación del gradiente consiste en plantear el problema como una optimización numérica en el que el objetivo es minimizar una función de coste añadiendo clasificadores débiles mediante el descenso del gradiente. Involucra tres elementos:

- La **función de coste** a optimizar: depende del tipo de problema a resolver.
- Un **clasificador débil** para hacer las predicciones: por lo general se usan árboles de decisión.
- Un **modelo que añade (ensambla) los clasificadores débiles para minimizar la función de coste**: se usa el descenso del gradiente para minimizar el coste al añadir árboles.

Los hiperparámetros más importantes que intervienen en este algoritmo (aunque no todos) son:

- **learning_rate**: determina el impacto de cada árbol en la salida final. Se parte de una estimación inicial que se va actualizando con la salida de cada árbol. Es el parámetro que controla la magnitud de las actualizaciones.
- **n_estimators**: número de clasificadores débiles a utilizar.

Como en este caso utilizaremos árboles de decisión como clasificadores débiles a ensamblar, también debemos tener en cuenta los hiperparámetros que afectan a esta clase de modelos. En este caso:

- **max_depth**: profundidad máxima del árbol.

Más información sobre el modelo que se utiliza en este ejemplo y de sus parámetros [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html).

In [8]:
# Creamos el modelo introduciendo los valores de los parámetros:
gb_clf = GradientBoostingClassifier(n_estimators=150, learning_rate=0.2, max_depth=3, random_state=0)

In [9]:
# Entrenamiento o ajuste del modelo con los datos de entrenamiento
gb_clf.fit(X_train_vec, y_train)

GradientBoostingClassifier(learning_rate=0.2, n_estimators=150, random_state=0)

In [10]:
# Predecimos sobre los datos de entrenamiento
pred_train = gb_clf.predict(X_train_vec)

# Mostramos el "classification report"
print('Resultados conjunto de entrenamiento:\n')
print(classification_report(y_train, pred_train))

Resultados conjunto de entrenamiento:

              precision    recall  f1-score   support

         ham       1.00      1.00      1.00       589
        spam       1.00      1.00      1.00       590

    accuracy                           1.00      1179
   macro avg       1.00      1.00      1.00      1179
weighted avg       1.00      1.00      1.00      1179



In [14]:
# Predecimos sobre los datos de test
pred_test = gb_clf.predict(X_test_vec)

# Mostramos el "classification report" y "accuracy"
accuracy = accuracy_score(y_test, pred_test)
print('Resultados conjunto de test:\n')
print(f'Accuracy: {accuracy:.2f}\n')
print(classification_report(y_test, pred_test))

Resultados conjunto de test:

Accuracy: 0.89

              precision    recall  f1-score   support

         ham       0.87      0.93      0.90       148
        spam       0.93      0.86      0.89       147

    accuracy                           0.89       295
   macro avg       0.90      0.89      0.89       295
weighted avg       0.90      0.89      0.89       295



### BLOQUE C: entrenamiento y inferencia con una red neuronal.

In [15]:
# Cargar el dataset (ajusta la ruta según el archivo que tengas)
df = pd.read_csv('../data/spam_filtered.csv')

# Verifica el dataset
df.head()

,target,text,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,ham,U dun say so early hor... U c already then say...,49
3,ham,"Nah I don't think he goes to usf, he lives aro...",61
4,spam,FreeMsg Hey there darling it's been 3 week's n...,148


In [16]:
# Preprocesamiento del texto
X = df['text']  # Supón que la columna 'text' contiene los mensajes
y = df['target']  # 'label' debe ser spam/no spam

In [17]:
# Convertir las etiquetas 'spam'/'ham' a números
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [29]:
# ¿Que valores asume la variale target transformada? 
y

array([0, 1, 0, ..., 0, 0, 0])

In [32]:
# ¿A qué corresponden los nuevos valores?
df['target'].values

array(['ham', 'spam', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [18]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y,random_state=42)

In [19]:
# Tokenización y padding para convertir el texto en secuencias de enteros
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


In [20]:
# ¿Come se transorma los textos con Tokenizer?
X_train_seq[0]

[5,
 370,
 813,
 5,
 746,
 69,
 2401,
 321,
 88,
 549,
 300,
 69,
 2402,
 2403,
 2404,
 321,
 99,
 814,
 2405,
 1893,
 196,
 116,
 370,
 815,
 18,
 2406,
 116,
 746,
 2407,
 65,
 5,
 2408,
 57,
 2409,
 1179,
 116,
 746,
 1894,
 16,
 57,
 76,
 18,
 691,
 116,
 371,
 116,
 370,
 1540,
 56,
 1180,
 80,
 1541,
 99,
 88,
 494,
 2410,
 1895,
 7,
 495,
 26,
 692,
 883,
 116,
 370,
 1333,
 1334,
 9,
 116,
 693,
 14,
 11,
 816,
 80,
 1896,
 116,
 371,
 1897,
 85,
 816,
 80,
 2411,
 2412,
 2413,
 2414,
 1898,
 1334,
 2415,
 1181,
 14,
 116,
 694,
 3,
 1335,
 47,
 695,
 116,
 306,
 817]

In [21]:
# Padding para asegurar que todas las secuencias tengan la misma longitud
X_train_pad = pad_sequences(X_train_seq, padding='post', maxlen=100)
X_test_pad = pad_sequences(X_test_seq, padding='post', maxlen=100)

Estructura de la red neuronal:
1. Capa de Embedding: 
    - input_dim=10000: Esto se refiere al tamaño del vocabulario (número total de palabras). 

    - output_dim=64: Este especifica la dimensión de los embeddings. En lugar de representar cada palabra con un valor binario o en un vector disperso (como ocurre con BoW o TF-IDF), las palabras se representarán por un vector denso de 64 dimensiones. 

    - input_length=100: se define la longitud máxima de las secuencias de entrada. Cada entrada de texto se convierte en una secuencia de índices de palabras (tokens), y si la longitud de una secuencia es menor que 100, se rellenará con ceros. Si es mayor, se truncará.
2. Capa de Pooling: Esta capa realiza un *pooling global promedio* sobre las secuencias de embeddings. Dado que las entradas son secuencias de longitud 100, esta capa tomará el promedio de los vectores de embeddings de las 100 palabras (tokens) en la secuencia, lo que reduce la representación de la secuencia de una matriz de 100x64 a un solo vector de 64 dimensiones

3. Capa densa (Fully connected): Esta capa toma la salida del pooling (un vector de 64 dimensiones) y lo pasa a través de una **capa densa con 64 neuronas** y la **función de activación ReLU**. Esto ayuda a la red a aprender características más complejas del texto procesado.

4. Capa de salida: esta capa tiene una sola neurona con la **función de activación Sigmoid**, que es adecuada para clasificación binaria (en este caso, para clasificar como spam o no spam). La salida de esta capa será un valor entre 0 y 1, que puedes interpretar como la probabilidad de que un mensaje sea spam.



In [26]:
# Modelo simple de red neuronal para clasificación de texto
nn_simple = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=100),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Para clasificación binaria (spam/no spam)
])

In [ ]:
# Compilar el modelo
nn_simple.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
nn_simple.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

In [ ]:
# Predecimos sobre los datos de test
pred_test = nn_simple.predict(X_test_pad)

26/26 [==============================] - 0s 2ms/step


In [ ]:
# ¿Qué contiene pred_test? ¿Qué son?
pred_test

array([[1.63494784e-04],
       [9.99683559e-01],
       [9.99902785e-01],
       [1.25901715e-04],
       [1.63715403e-03],
       [3.20897717e-03],
       [1.99610298e-03],
       [4.88012331e-03],
       [9.99916255e-01],
       [1.52244326e-03],
       [5.41562820e-03],
       [7.90504813e-01],
       [9.99728560e-01],
       [1.01553982e-04],
       [8.08721161e-05],
       [1.35890897e-02],
       [5.92035009e-04],
       [6.29816859e-05],
       [9.98020172e-01],
       [4.49418367e-05],
       [3.18931462e-03],
       [7.11744884e-03],
       [6.83361478e-03],
       [9.99737680e-01],
       [2.80932756e-03],
       [4.34677536e-03],
       [5.57146817e-02],
       [9.99936640e-01],
       [1.73422508e-02],
       [5.21538139e-04],
       [3.72867973e-04],
       [7.31132575e-04],
       [9.66990113e-01],
       [9.72566247e-01],
       [9.98693287e-01],
       [9.99632239e-01],
       [3.23799141e-02],
       [2.94691529e-02],
       [4.27135004e-04],
       [1.85513988e-01],


In [36]:
# Convertir las predicciones a clases (spam o no spam)
# La salida será un valor entre 0 y 1 (probabilidad de spam)
predicted_labels = (pred_test > 0.5).astype(int)  # Clasifica como spam (1) si la probabilidad > 0.5, sino no spam (0)
predicted_labels

array([[0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
    

In [37]:
# Mostramos el "classification report" y "accuracy"
accuracy = accuracy_score(y_test, predicted_labels)
print('Resultados conjunto de test:\n')
print(f'Accuracy: {accuracy:.2f}\n')
print(classification_report(y_test, predicted_labels))

Resultados conjunto de test:

Accuracy: 0.98

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       659
           1       0.97      0.94      0.95       147

    accuracy                           0.98       806
   macro avg       0.98      0.97      0.97       806
weighted avg       0.98      0.98      0.98       806



### Red Neuronal Compleja

### Transfer Learning con un Modelo Preentrenado

DistilBERT (una versión más ligera de BERT)

In [ ]:
# Cargar el tokenizer de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
# Tokenización de los textos
X_train_enc = tokenizer(list(X_train), padding=True, truncation=True, max_length=100, return_tensors='tf')
X_test_enc = tokenizer(list(X_test), padding=True, truncation=True, max_length=100, return_tensors='tf')

# Cargar el modelo preentrenado DistilBERT para clasificación de texto
model_bert = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Compilar el modelo
model_bert.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento con transfer learning
model_bert.fit(X_train_enc['input_ids'], y_train, epochs=3, batch_size=128, validation_data=(X_test_enc['input_ids'], y_test))

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/3
16/26 [=================>............] - ETA: 4:40 - loss: 1.1237 - accuracy: 0.7778